## Clasificación TF-IDF

In [1]:
import pandas as pd
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess

Lo primero que necesitamos es cargar nuestros glosarios de términos para crear nuestro diccionario

In [2]:
def cargar_glosario(categoria):
    fname = f"../Datos/glosario_{categoria}.txt"
    glosario = []
    with open(fname, 'r') as f:
        glosario = [termino.rstrip('\n') for termino in f.readlines()]
    return glosario

In [3]:
categorias = ["deportes", "salud", "ciencia", "politica"]
glosarios = {}
for categoria in categorias:
    glosarios[categoria] = cargar_glosario(categoria)

Creamos nuestro diccionario de palabras en base a los terminos del glosario

In [4]:
def create_dictionary(glosarios):
    doc_tokens = [[termino for termino in glosario] for glosario in glosarios.values()]
    dictionary = corpora.Dictionary(doc_tokens)
    return dictionary

In [5]:
glosarios_dict = create_dictionary(glosarios)
print(glosarios_dict)

Dictionary<200 unique tokens: ['aerodinamico', 'atletico', 'booker', 'boston', 'cambio']...>


Cargamos nuestras noticias de test y las convertimos a un bag of words utilizando nuestro diccionario

In [6]:
noticias_test_dataframe = pd.read_csv("../Datos/noticias_test.csv")

In [7]:
def create_bag_of_words(docs_list, dictionary):
    doc_tokens = [simple_preprocess(corpus) for corpus in docs_list]
    docs_bow = [dictionary.doc2bow(doc) for doc in doc_tokens]
    return docs_bow

In [8]:
def create_tfidf(docs_bow):
    docs_tfidf = models.TfidfModel(docs_bow, smartirs="lfc")
    return docs_tfidf

Ahora es momento de crear una bag of words para cada noticia en base a nuestro diccionario

In [9]:
docs_bow = create_bag_of_words(noticias_test_dataframe["corpus"].values, glosarios_dict)
print(docs_bow[19])

[(7, 1), (17, 2), (38, 1), (70, 1)]


Y con el bag of words podemos crear nuestro modelo tfidf

In [10]:
tfidf_model = create_tfidf(docs_bow)
tfidf_model[docs_bow[0]]

[(1, 0.5047168663250167), (27, 0.8632849383877002)]

Representamos nuestros glosarios en tfidf

In [11]:
deportes_bow = glosarios_dict.doc2bow(glosarios["deportes"])
deportes_tfidf = tfidf_model[deportes_bow]

salud_bow = glosarios_dict.doc2bow(glosarios["salud"])
salud_tfidf = tfidf_model[salud_bow]

ciencia_bow = glosarios_dict.doc2bow(glosarios["ciencia"])
ciencia_tfidf = tfidf_model[ciencia_bow]

politica_bow = glosarios_dict.doc2bow(glosarios["politica"])
politica_tfidf = tfidf_model[politica_bow]

Ahora clasificamos

In [12]:
index = similarities.MatrixSimilarity(tfidf_model[docs_bow])
similarities_deportes = index[deportes_tfidf]
similarities_salud = index[salud_tfidf]
similarities_ciencia = index[ciencia_tfidf]
similarities_politica = index[politica_tfidf]

In [13]:
def get_prediction(index, 
                   similarities_deportes,       
                   similarities_salud,                       
                   similarities_ciencia,
                   similarities_politica):
    mejor_match = 0
    mejor = "No clasificado"
    if (similarities_deportes[index] > mejor_match):
        mejor_match = similarities_deportes[index]
        mejor = "sports"
    if (similarities_salud[index] > mejor_match):
        mejor_match = similarities_salud[index]
        mejor = "health"
    if (similarities_ciencia[index] > mejor_match):
        mejor_match = similarities_ciencia[index]
        mejor = "science"
    if (similarities_politica[index] > mejor_match):
        mejor_match = similarities_politica[index]
        mejor = "politics"
    return mejor
    

In [14]:
predictions = []
for i in range(len(docs_bow)):
    pred = get_prediction(i, 
                          similarities_deportes,
                          similarities_salud,
                          similarities_ciencia,
                          similarities_politica)
    predictions.append(pred)

In [15]:
noticias_test_dataframe.insert(2, "labels", predictions)

In [16]:
noticias_test_dataframe.to_csv('a.csv')

In [17]:
noticias = pd.read_csv("../Datos/urls.csv")
fuentes = []
for tit in noticias["title"].values:
    t  = tit.split("-")[-1]
    fuentes.append(t)
print(len(set(fuentes)))
for f in set(fuentes):
    print("            \item", f)

65
            \item  ATP Tour
            \item  Eurosport ES
            \item  Antena 3 Noticias
            \item  El Periódico
            \item  NotiPress
            \item  El Imparcial de Oaxaca
            \item  Redacción Médica
            \item  INFORMACIÓN
            \item  asajajaen.com
            \item  Alimente
            \item  Bernabéu Digital
            \item  La Razón
            \item  Noticias de Gipuzkoa
            \item  Onda Cero
            \item  Vitónica
            \item  MUI Kitchen
            \item  Planeta Triatlón
            \item  AS 
            \item  Newtral
            \item  Superdeporte
            \item  Defensa Central
            \item  Car and Driver 
            \item  ES
            \item  MARCA.com
            \item  El Mundo
            \item  EL ESPAÑOL
            \item  Faro de Vigo
            \item  Diario de Ávila
            \item  laSexta
            \item  Sporting News
            \item  TRT Español
            \item  MSN